<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/211229_sm_know_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model with only 2017 data

In [1]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-kpqgzj92/kobert-tokenizer_0ea010ce072b4491937807bea0d836f2
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-kpqgzj92/kobert-tokenizer_0ea010ce072b4491937807bea0d836f2
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6es9fhi7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6es9fhi7


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (82,95,98,109,111,113,127,128,129,132,136) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (93,94,97,98,99,100,104,105,107,112,132,136,140,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (179) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (98,109,111,115,117,128,132) have mixed types.Specify dtype option on import or set 

In [5]:
#bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1
texts = ['bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in texts:
  train1[col] = train1[col].map(null_to_no)

cols = train1.columns[1:-1].drop(texts)

train1[cols] = train1[cols].astype(str)

cols = train1.columns[1:-1]

text = ''

for col in cols:
  text += train1[col] + ' '

train1['text'] = text

In [6]:
# bq28_1	bq29	bq30	bq31	bq32  bq37_1
texts = ['bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in texts:
  train2[col] = train2[col].map(null_to_no)

cols = train2.columns[1:-1].drop(texts)

train2[cols] = train2[cols].astype(str)

cols = train2.columns[1:-1]

text = ''

for col in cols:
  text += train2[col] + ' '

train2['text'] = text

In [7]:
# bq18_10	bq20_1 bq22	bq23	bq24 bq27_1
texts = ['bq18_10',	'bq20_1',	'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in texts:
  train3[col] = train3[col].map(null_to_no)

cols = train3.columns[1:-1].drop(texts)

train3[cols] = train3[cols].astype(str)

cols = train3.columns[1:-1]

text = ''

for col in cols:
  text += train3[col] + ' '

train3['text'] = text

In [8]:
# bq18_10 bq20_1 bq26_1
texts = ['bq18_10',	'bq20_1',	'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in texts:
  train4[col] = train4[col].map(null_to_no)

cols = train4.columns[1:-1].drop(texts)

train4[cols] = train4[cols].astype(str)

cols = train4.columns[1:-1]

text = ''

for col in cols:
  text += train4[col] + ' '

train4['text'] = text

In [9]:
df = pd.concat([train1[['idx', 'text', 'knowcode']], train2[['idx', 'text', 'knowcode']], train3[['idx', 'text', 'knowcode']], train4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)

In [10]:
df['text'] = df['text'].astype(str)

In [11]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

def remove_tab(text:str):
  return text.replace('\t', ' ')

df['text'] = df['text'].map(remove_tab)

def remove_comma(text:str):
  return text.replace(',', ' ')

df['text'] = df['text'].map(remove_comma)

In [12]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [13]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [14]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [15]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['knowcode'].map(target_to_label)

In [16]:
dataset_train, dataset_test = train_test_split(df[['idx', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [17]:
df

,idx,text,knowcode,target_label
0,0,3 3 3 3 3 3 4 4 3 4 3 3 2 2 2 2 2 2 3 3 3 3 4 ...,825101,0
1,1,4 5 4 5 3 4 3 4 3 4 1 1 1 1 2 3 3 4 3 ...,140204,1
2,2,3 4 3 4 3 4 5 6 4 5 1 1 3 4 3 4 3 4 1 3 ...,140204,1
3,3,3 3 3 3 3 5 4 5 4 6 3 5 3 4 4 5 5 5 4 5 3 4 4 ...,140601,2
4,4,4 5 3 4 3 4 4 5 3 4 1 1 1 1 3 4 2 2 3 ...,140204,1
...,...,...,...,...
35228,35244,4 6 3 5 3 3 1 5 5 2 3 1 3 3 4 4 3 5 2 4 1 ...,121105,378
35229,35245,3 4 4 5 2 4 4 6 1 2 4 5 5 3 4 4 4 2 3 3 3 3 ...,26102,414
35230,35246,5 6 3 4 3 4 4 5 5 7 4 6 1 2 2 2 4 2 2 2 2 1 ...,31301,336
35231,35247,5 6 5 5 4 5 3 3 3 4 4 5 3 4 5 5 5 6 4 5 3 5 3 ...,154105,584


In [18]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [19]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [21]:
## Setting parameters
max_len = 512
batch_size = 4
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [23]:
next(iter(data_train))

(array([   2,  611,  529,  605,  605,  605,  605,  605,  589,  605,  553,
         589,  589,  589,  605,  605,  605,  589,  605,  517,   54,  526,
         605,  517,   54,  526,  529,  517,   54,  526,  605,  517,   54,
         526,  529,  517,   54,  526,  529,  517,   54,  526,  605,  517,
          54,  526,  529,  517,   54,  526,  605,  517,   54,  526,  529,
         517,   54,  526,  605,  517,   54,  526,  553,  517,   54,  526,
         553,  517,   54,  526,  553,  517,   54,  526,  553,  517,   54,
         526,  553,  517,   54,  526,  605,  517,   54,  526,  605,  517,
          54,  526,  553,  517,   54,  526,  605,  517,   54,  526,  605,
         517,   54,  526,  605,  517,   54,  526,  589,  517,   54,  526,
         611,  517,   54,  526,  589,  517,   54,  526,  611,  517,   54,
         526,  529,  517,   54,  526,  605,  517,   54,  526,  605,  517,
          54,  526,  529,  517,   54,  526,  529,  517,   54,  526,  529,
         517,   54,  526,  529,  517, 

In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [25]:
len(df['target_label'].unique())

585

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [30]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [31]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
train = []
test = []
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train.append(train_acc / (batch_id+1))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test.append(test_acc / (batch_id+1))
    # torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/7047 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.010088920593262 train acc 0.0
epoch 1 batch id 201 loss 6.14228630065918 train acc 0.0024875621890547263
epoch 1 batch id 401 loss 6.625829696655273 train acc 0.0012468827930174563
epoch 1 batch id 601 loss 6.363317489624023 train acc 0.0008319467554076539
epoch 1 batch id 801 loss 6.354059219360352 train acc 0.0009363295880149813
epoch 1 batch id 1001 loss 6.215842247009277 train acc 0.0012487512487512488
epoch 1 batch id 1201 loss 6.34815788269043 train acc 0.0012489592006661116
epoch 1 batch id 1401 loss 6.416350841522217 train acc 0.0014275517487508922
epoch 1 batch id 1601 loss 6.481693744659424 train acc 0.0023422860712054966
epoch 1 batch id 1801 loss 6.014464378356934 train acc 0.003192670738478623
epoch 1 batch id 2001 loss 6.207036972045898 train acc 0.004122938530734633
epoch 1 batch id 2201 loss 6.335580825805664 train acc 0.004429804634257156
epoch 1 batch id 2401 loss 6.461093425750732 train acc 0.004373177842565598
epoch 1 batch id 2601 loss 6.2

  0%|          | 0/1762 [00:00<?, ?it/s]

epoch 1 test acc 0.01007377979568672


...

  0%|          | 0/1762 [00:00<?, ?it/s]

epoch 2 test acc 0.01007377979568672


  0%|          | 0/7047 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 6.48012638092041 train acc 0.0
epoch 3 batch id 201 loss 6.486926555633545 train acc 0.011194029850746268
epoch 3 batch id 401 loss 6.388457775115967 train acc 0.011845386533665835
epoch 3 batch id 601 loss 5.8643951416015625 train acc 0.012063227953410981
epoch 3 batch id 801 loss 6.459906578063965 train acc 0.011548064918851436
epoch 3 batch id 1001 loss 6.3826799392700195 train acc 0.011738261738261738
epoch 3 batch id 1201 loss 6.439223289489746 train acc 0.011656952539550375
epoch 3 batch id 1401 loss 6.61870002746582 train acc 0.011955745895788723
epoch 3 batch id 1601 loss 6.549194812774658 train acc 0.012648344784509682
epoch 3 batch id 1801 loss 6.256855010986328 train acc 0.01263187118267629
epoch 3 batch id 2001 loss 6.49863862991333 train acc 0.012993503248375811
epoch 3 batch id 2201 loss 6.465415954589844 train acc 0.012721490231712857
epoch 3 batch id 2401 loss 6.314047336578369 train acc 0.013015410245730946
epoch 3 batch id 2601 loss 6.243819713

KeyboardInterrupt: ignored

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/know/model/model.pt')